# Speech Recognition II

Este código implementa un modelo de red neuronal recurrente utilizando Keras, específicamente para la tarea de clasificación de sentimientos en el conjunto de datos IMDB

In [3]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 104.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 83.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

In [7]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parámetros del modelo
max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto
embedding_dim = 128  # Dimensión de los vectores de palabra de la capa Embedding

# Cargar el conjunto de datos IMDB
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Preprocesamiento de datos
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

# Crear el modelo
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=256, return_sequences=True))  # Añade return_sequences=True aquí
model.add(Dropout(0.2))
model.add(LSTM(units=128))  # Esta segunda capa LSTM ahora recibirá la entrada correcta
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train, batch_size=64, epochs=3, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Pérdida del modelo: {loss:.4f}")
print(f"Precisión del modelo: {accuracy*100:.2f}%")
model.save()

Epoch 1/3
313/313 [==============================] - 304s 960ms/step - loss: 0.4771 - accuracy: 0.7685 - val_loss: 0.4146 - val_accuracy: 0.8172
Epoch 2/3
313/313 [==============================] - 305s 974ms/step - loss: 0.3335 - accuracy: 0.8585 - val_loss: 0.3894 - val_accuracy: 0.8552
Epoch 3/3
782/782 [==============================] - 174s 223ms/step - loss: 0.3361 - accuracy: 0.8633
Pérdida del modelo: 0.3361
Precisión del modelo: 86.33%


TypeError: Model.save() missing 1 required positional argument: 'filepath'

Se guarda el modelo

In [8]:
model.save('sentimientos.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
Cada reeseña se convierte en una secuencia numérica, donde cada palabra se reemplaza por su índice numérico correspondiente en word_index

In [11]:
# Obtener el word_index del conjunto de datos IMDB
word_index = imdb.get_word_index()

# Realizar predicciones
sample_review = "Esta película es genial, realmente la disfruté."
# Preprocesar la revisión de muestra y convertirla en una secuencia numérica
sample_sequence = [word_index.get(word, 0) if word_index.get(word, 0) < max_features else 0 for word in sample_review.split()]
sample_sequence = pad_sequences([sample_sequence], maxlen=max_len)

# Realizar la predicción
prediction = model.predict(sample_sequence)[0][0]
sentiment = "positivo" if prediction >= 0.5 else "negativo"
print(f"Predicción de sentimiento: {sentiment} (Confianza: {prediction:.2f})")


1/1 [==============================] - 0s 94ms/step
Predicción de sentimiento: negativo (Confianza: 0.33)


Se interpreta la salida del modelo: un valor de predicción cercano a 1 sugiere un sentimiento positivo, 
mientras que un valor cercano a 0 indica un sentimiento negativo. Se utiliza un umbral de 0.5 para clasificar 
la predicción como 'positivo' o 'negativo'.

In [13]:
# Parámetros del modelo
max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto
# Cargar el word_index
word_index = imdb.get_word_index()
# Función para convertir texto a secuencia
def text_to_sequence(text, word_index, max_features, max_len):
    # Convertir a minúsculas y tokenizar
    tokens = text.lower().split()
    # Convertir tokens a índices
    sequence = [word_index.get(word, 0) if word_index.get(word, 0) < max_features else 0 for word in tokens]
    # Acolchar la secuencia
    padded_sequence = pad_sequences([sequence], maxlen=max_len)
    return padded_sequence

# Texto de entrada
#sample_review = "this candy is bitter I don't like it"
sample_review = "very delicious the food in this restaurant"
# Convertir el texto en secuencia
sample_sequence = text_to_sequence(sample_review, word_index, max_features, max_len)

# Realizar la predicción
prediction = model.predict(sample_sequence)[0][0]
sentiment = "positivo" if prediction >= 0.5 else "negativo"
print(f"Predicción de sentimiento: {sentiment} (Confianza: {prediction:.2f})")

1/1 [==============================] - 0s 87ms/step
Predicción de sentimiento: positivo (Confianza: 0.59)


visualizando la secuencia que fue convertida el texto ingresado

In [14]:
print(sample_sequence)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   52   

cargando el modelo guardado y realizando al prediccion

In [16]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Cargar el modelo
model_load = load_model('sentimientos.h5')

max_features = 5000  # Número máximo de palabras a considerar como características
max_len = 200  # Longitud máxima de la secuencia de texto

# Cargar el word_index
word_index = imdb.get_word_index()

# Función para convertir texto a secuencia
def text_to_sequence(text, word_index, max_features, max_len):
    tokens = text.lower().split()
    sequence = [word_index.get(word, 0) if word_index.get(word, 0) < max_features else 0 for word in tokens]
    padded_sequence = pad_sequences([sequence], maxlen=max_len)
    return padded_sequence

# Texto de entrada para la predicción
sample_review = "this movie is bitter I don't like it"

# Convertir el texto en secuencia
sample_sequence = text_to_sequence(sample_review, word_index, max_features, max_len)

# Realizar la predicción
prediction = model_load.predict(sample_sequence)[0][0]
sentiment = "positivo" if prediction >= 0.5 else "negativo"
print(f"Predicción de sentimiento: {sentiment} (Confianza: {prediction:.2f})")

1/1 [==============================] - 1s 749ms/step
Predicción de sentimiento: negativo (Confianza: 0.30)
